# 기본 라이브러리

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import requests
import os
from glob import glob
import io
import numpy as np
from io import BytesIO

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max.colwidth',100)
pd.options.display.max_columns = 300

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Train image features extraction

In [2]:
df = pd.read_csv('small_total_train_set.csv', encoding='utf-8-sig')
df.head()
df.shape

,Unnamed: 0,Unnamed: 0.1,channel_name,subscribers,video_name,days_after_upload,video_duration,hits,likes_num,dislikes_num,category_id,thumbnail
0,0,0,복덩이,104000,"너무나맛있는 '알타리묵은지찜과' ""앞치마에그림"" Altari old paper steamed ' And 'picture in apron'",10,311,1126,40,1,0,https://i.ytimg.com/vi/al52wJgMGyI/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ=...
1,1,1,복덩이,104000,헌옷으로만든 [앞치마]와 '민들레' 겉절이 'Apron' made with old clothes and a beautiful life,11,416,1067,85,3,0,https://i.ytimg.com/vi/-xx7shznKec/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ=...
2,2,2,복덩이,104000,탈모와다이어트에 탁월한고급진 '느타리버섯볶음' 과 프로방스 창문그리기 Stir-fried 'Oyster Mushroom' and drawing Provence window,12,265,860,64,0,0,https://i.ytimg.com/vi/m7KC7oEKKuo/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ=...
3,3,3,복덩이,104000,감기 면연력 간기능보호에 탁월한 '도라지튀김' '민들레튀김' '라벤다그리기' Fried bellflower Dandelion fried lavender,13,419,1116,76,1,0,https://i.ytimg.com/vi/FYeuMGEgNWA/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ=...
4,4,4,복덩이,104000,진달래화전 부침 복덩이표 바삭바삭한 쫀득쫀득비법공개,14,287,622,47,2,0,https://i.ytimg.com/vi/UjLKW2fjkyA/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ=...


(13685, 12)

In [4]:
category = df['category_id']
thumbnail = df['thumbnail']

- 이미지 변환

In [5]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input

Using TensorFlow backend.


In [17]:
image_w = 64
image_h = 64
pixels = image_w * image_h * 3
X = []
for i in thumbnail :
    
    url = i+str('.jpg')
    response = requests.get(url, stream=True) 
    img= Image.open(io.BytesIO(response.content)) 
    
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    img = image.img_to_array(img)

    # 데이터를 전처리합니다(채널별 컬러 정규화를 수행합니다)
    img = preprocess_input(img)
    X.append(img)

In [44]:
X = np.array(X)
Y = np.array(category)
Y = pd.get_dummies(Y).to_numpy()
X.shape

(13685, 64, 64, 3)

In [37]:
np.save('total_train_X_imgs',X)

- 모델 구성

In [38]:
# 카테고리 지정하기
categories = category.unique()
nb_classes = len(categories)
# 이미지 크기 지정하기
image_w = 64
image_h = 64

# 데이터 정규화하기(0~1사이로)
X = X.astype("float") / 64 # 256 으로 하면?
print('X shape:', X.shape) # 74%

X shape: (13685, 64, 64, 3)


In [39]:
from keras.models import Sequential
from keras import metrics
from keras import optimizers
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
import os

In [40]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=X.shape[1:], padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3))) #전체 train에 돌릴때는 32로
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
last_layer = model.add(Dropout(0.25))

# 전결합층
model.add(Flatten())    # 벡터형태로 reshape
model.add(Dense(512))   # 출력
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))
# 모델 구축하기
model.compile(loss='categorical_crossentropy',   # 최적화 함수 지정
    optimizer='rmsprop',
    metrics=['accuracy'])

# 모델 요약
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 30, 32)      

- flatten_layer 추출

In [41]:
from keras.models import Model
layer_name = 'flatten_1'
flatten_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
flatten_output = flatten_layer_model.predict(X)

In [42]:
flatten_output.shape

(13685, 7200)

In [19]:
seq_image_features = pd.DataFrame(flatten_output)
seq_image_features.to_csv('image_features_유경.csv')

TypeError: to_csv() got an unexpected keyword argument 'index_col'

In [22]:
a = pd.read_csv('extracted_image_features_유경.csv')
np.save('extracted_image_features_selfcnn',a)

# Test image feature extraction

In [2]:
df = pd.read_csv('small_total_test_set.csv', encoding='utf-8-sig')
df.head()
df.shape

,Unnamed: 0,Unnamed: 0.1,channel_name,subscribers,video_name,days_after_upload,video_duration,hits,likes_num,dislikes_num,category_id,thumbnail
0,9317,9317,얌무YAMMoo,101000,내가 만든 대왕 버터 먹방ㅣ봘움이꿔여무ㅣYAMMoo,89,485,657250,11000,289,0,https://i.ytimg.com/vi/xPPcUjs6DZE/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ=...
1,9318,9318,얌무YAMMoo,101000,"내가 만든 대왕 망한 치즈 덩어리,,,ㅣ주절주절무ㅣㅣYAMMoo",98,341,827802,1000,246,0,https://i.ytimg.com/vi/YN7CeXQ5gw0/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ=...
2,9319,9319,얌무YAMMoo,101000,내가 만든 대왕 아이셔 먹방ㅣ잉차잉차무ㅣYAMMoo,110,516,1139203,18000,925,0,https://i.ytimg.com/vi/oGPOIoYpYv4/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ=...
3,9320,9320,얌무YAMMoo,101000,"내가 만든 이불,, 아니 대왕 질러 육포 먹방ㅣ카멜레온무ㅣYAMMoo",117,558,1071542,16000,558,0,https://i.ytimg.com/vi/x_CYbdVd6K8/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ=...
4,9321,9321,얌무YAMMoo,101000,내가 만든 대왕 짜요짜요 먹방ㅣ피부에 수분을 얻었무ㅣYAMMoo,122,459,1072883,17000,742,0,https://i.ytimg.com/vi/Ylb1VzQm-GQ/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ=...


(3500, 12)

In [3]:
category = df['category_id']
thumbnail = df['thumbnail']

In [4]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
image_w = 64
image_h = 64
pixels = image_w * image_h * 3
TestX = []
for i in thumbnail :
    
    url = i+str('.jpg')
    response = requests.get(url, stream=True) 
    img= Image.open(io.BytesIO(response.content)) 
    
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    img = image.img_to_array(img)

    # 데이터를 전처리합니다(채널별 컬러 정규화를 수행합니다)
    img = preprocess_input(img)
    TestX.append(img)

Using TensorFlow backend.


In [5]:
TestX = np.array(TestX)
TestY = np.array(category)
TestY = pd.get_dummies(TestY).to_numpy()
TestX.shape

(3500, 64, 64, 3)

In [8]:
np.save('total_test_X_imgs',TestX)

In [9]:
# 카테고리 지정하기
categories = category.unique()
nb_classes = len(categories)
# 이미지 크기 지정하기
image_w = 64
image_h = 64

# 데이터 정규화하기(0~1사이로)
TestX = TestX.astype("float") / 64 # 256 으로 하면?
print('X shape:', TestX.shape) # 74%

X shape: (3500, 64, 64, 3)


In [10]:
from keras.models import Sequential
from keras import metrics
from keras import optimizers
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
import os

In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=TestX.shape[1:], padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3))) #전체 train에 돌릴때는 32로
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
last_layer = model.add(Dropout(0.25))

# 전결합층
model.add(Flatten())    # 벡터형태로 reshape
model.add(Dense(512))   # 출력
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))
# 모델 구축하기
model.compile(loss='categorical_crossentropy',   # 최적화 함수 지정
    optimizer='rmsprop',
    metrics=['accuracy'])

# 모델 요약
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 30, 32)      

In [12]:
from keras.models import Model
layer_name = 'flatten_1'
flatten_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
flatten_output = flatten_layer_model.predict(TestX)

In [27]:
pd.DataFrame(flatten_output)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,...,7050,7051,7052,7053,7054,7055,7056,7057,7058,7059,7060,7061,7062,7063,7064,7065,7066,7067,7068,7069,7070,7071,7072,7073,7074,7075,7076,7077,7078,7079,7080,7081,7082,7083,7084,7085,7086,7087,7088,7089,7090,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129,7130,7131,7132,7133,7134,7135,7136,7137,7138,7139,7140,7141,7142,7143,7144,7145,7146,7147,7148,7149,7150,7151,7152,7153,7154,7155,7156,7157,7158,7159,7160,7161,7162,7163,7164,7165,7166,7167,7168,7169,7170,7171,7172,7173,7174,7175,7176,7177,7178,7179,7180,7181,7182,7183,7184,7185,7186,7187,7188,7189,7190,7191,7192,7193,7194,7195,7196,7197,7198,7199
0,-0.001158,0.002141,0.008723,-0.000788,0.004164,-0.001598,0.000216,-7.286071e-04,0.001323,-0.003510,-0.001297,0.003609,0.005785,-0.001276,0.007693,0.006266,0.006742,-0.006156,-0.001629,-0.002857,0.000551,0.009636,0.001019,0.004807,0.000550,-0.001277,0.008686,0.000231,0.007196,-0.006865,-0.004347,0.000977,-0.000950,0.002810,0.006892,-0.004122,0.004517,-0.001335,0.000319,-0.002970,0.001419,-0.002978,0.000148,0.002290,0.004179,-0.000988,0.008317,0.006144,0.008091,-0.007911,-0.001618,-0.002228,0.000492,0.010128,0.001566,0.003848,0.000265,-0.001282,0.009347,0.000046,0.006660,-0.008168,-0.005429,-0.001545,-0.000424,0.002983,0.006849,-0.004329,0.004599,-0.001290,0.000630,-0.002795,0.001307,-0.002710,-0.000150,0.002496,0.004063,-0.001355,0.008430,0.006213,0.008188,-0.007757,-0.001600,-0.002127,0.000546,0.009953,0.001753,0.004043,0.000333,-0.001825,0.009878,-0.000429,0.006689,-0.008112,-0.005661,-0.001605,-0.000173,0.003114,0.006865,-0.004416,0.004664,-0.001308,0.000764,-0.002779,0.001165,-0.002662,-7.752990e-05,0.002592,0.003980,-0.001453,0.008494,0.005976,0.008265,-0.007617,-0.001559,-0.001965,0.000609,0.009899,0.001824,0.004231,0.000395,-0.001807,0.009963,-0.000528,0.006619,-0.008036,-0.005802,-0.001675,-0.000217,0.003126,0.006963,-0.004451,0.004700,-0.001295,0.000790,-0.002740,0.001165,-0.002689,-0.000165,0.002647,0.003977,-0.001485,0.008570,0.005971,0.008352,-0.007616,-0.001585,-0.002008,0.000612,0.009697,...,0.001032,0.001006,0.003000,-0.000456,0.005066,0.005603,0.002827,-0.008463,-0.001736,-0.001431,-0.001078,0.007534,0.001853,0.001034,0.001549,-0.001160,0.006744,0.001691,0.006396,-0.008291,-0.003738,0.002416,-0.002325,0.001926,0.007703,-0.003561,0.004841,-0.000938,-0.002413,-0.005237,0.002943,0.000987,0.001064,0.000967,0.003707,-0.000448,0.005112,0.005590,0.002930,-0.008477,-0.001584,-0.001557,-0.002075,0.007879,0.000849,0.001424,0.001058,-0.001067,0.006754,0.001726,0.006355,-0.008673,-0.003811,0.002504,-0.002196,0.002016,0.008307,-0.003539,0.003807,-0.000974,-0.002405,-0.004750,0.002787,0.000782,0.001086,0.001146,0.003664,-0.000300,0.005070,0.005845,0.003013,-0.009348,-0.001022,-0.001621,-0.000993,0.007496,0.000554,0.000694,0.002110,-0.000889,0.006911,0.001624,0.006116,-0.008592,-0.003777,0.002182,-0.000857,0.003330,0.007838,-0.003645,0.004129,-0.000216,-0.002155,-0.005101,0.002250,-0.000171,0.001135,0.000964,0.003417,-0.000198,0.005279,0.006068,0.003774,-0.009175,-0.001140,-0.001682,-0.001103,0.007022,0.003426,0.001281,0.001227,-0.001061,0.007695,0.001650,0.006122,-0.007935,-0.004448,0.001080,-0.002787,0.001940,0.007638,-0.002566,0.005548,-0.001229,-0.001085,-0.004560,0.000566,-0.000874,0.002311,0.001442,0.004791,0.000691,0.006493,0.007258,0.003873,-0.007033,0.000432,-0.000646,0.000543,0.008097,0.000463,

In [23]:
np.save('@extracted_test_image_features_유경',flatten_output)